# Loan Eligibility Prediction System

we are going to build a machine learning model which is going to decide whether a loan can be granted or not by predicting whether the loan requester can repay the loan or not. The model is going to decide this by looking at a majority of parameters.

## Importing The Required Packages

We will now import our required packages for building this system. They are :-

1. NumPy
2. Scikit-Learn
3. Pandas
4. Seaborn

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## Getting The Data

We have got our required dataset from [Analytics Vidhya](https://www.analyticsvidhya.com/).

The dataset contains the following features :-

* Loan_ID - The ID number of the Loan application
* Gender - The Gender of the applicant
* Married - Whether the applicant is married or not
* Dependents - How many is dependant on the applicant financially
* Education  - What is the Highest Education Qualification of the Applicant
* Self_Employed - Is the Applicant self-employed?
* ApplicantIncome - What is the Applicant's income?
* CoapplcantIncome - What is the Co-Applicant's income?
* LoanAmount - What is the principal loan amount?
* Loan_Amount_Term - What is the loan term?
* Credit_History - The history of Applicant's credit
* Property_Areas - Where are the Applicant's collateral property located?
* Loan_Status - Should the loan be approved or not? (target feature)
